In [56]:
import pandas as pd
import sqlite3
import codecs

file_path = 'users1.csv'
with codecs.open(file_path, 'r', encoding='koi8_r') as file:
    user = pd.read_csv(file, sep='\t', header=None, names=['user_id', 'email', 'geo'])

file_path = 'log1.csv'
log = pd.read_csv(file_path, header=None, names=['user_id', 'time', 'bet', 'win'])


conn = sqlite3.connect('bets.s3db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE LOG
(user_id VARCHAR(50) NOT NULL, 
time DATE, 
bet VARCHAR(50),
win VARCHAR(50));
''')

cursor.execute('''
CREATE TABLE USERS
(user_id VARCHAR(50) NOT NULL,
email VARCHAR(50),
geo VARCHAR(50));
''')



for index, row in log.iterrows():
    cursor.execute('''
    INSERT INTO LOG (user_id, time, bet, win) VALUES (?, ?, ?, ?)
    ''', (row['user_id'], row['time'], row['bet'], row['win']))

for index, row in df.iterrows():
    cursor.execute('''
    INSERT INTO USERS (user_id, email, geo) VALUES (?, ?, ?)
    ''', (row['user_id'], row['email'], row['geo']))

conn.commit()
conn.close()


In [ ]:
CREATE TABLE LOG
(user_id VARCHAR(50) NOT NULL, 
time DATE, 
bet VARCHAR(50),
win VARCHAR(50));

CREATE TABLE USERS
(user_id VARCHAR(50) NOT NULL,
email VARCHAR(50),
geo VARCHAR(50));

In [ ]:
--7) Запросы для чистки грязных данных (выполнять можно в любой последовательности): 
UPDATE LOG SET user_id  = substring(user_id, 30, 4) where user_id LIKE 
'%апись пользователя № - user_%';
--Оставляем только id без слова user

DELETE from LOG where user_id like '#error%';
--Для чистки данных со значением #error

DELETE from USERS where user_id not like '%User%';
--Для чистки хедера в таблице USERS

UPDATE USERS SET user_id  = substring(user_id, 5, 4);
--Оставляем только id без слова user

UPDATE LOG SET time  = substring(time, 2) where time LIKE '[%';
UPDATE LOG SET time = substring(time, 1, 11)||'0'||substring(time,12) where LENGTH(time) = 18;
--Обрабатываем время для работыв в задачах

--8) ЗАДАЧИ

SELECT MIN(count_attempts) from (SELECT user_id, COUNT(time) as count_attempts from (SELECT * from (LOG inner join (SELECT user_id, MIN(time) as min_time from LOG where bet != '' GROUP BY user_id) as A on LOG.user_id=A.user_id)) where time <= min_time GROUP BY user_id);

SELECT AVG(percent_win) FROM (SELECT geo, (100*(win/bet)) as percent_win FROM LOG inner join USERS on LOG.user_id = USERS.user_id and bet != '') where percent_win > 0;

SELECT user_id, (SUM(win) - SUM(bet)) as balance from LOG GROUP BY user_id

SELECT geo FROM(SELECT geo, MAX(100*(win/bet)) as percent_win FROM LOG inner join USERS on LOG.user_id = USERS.user_id and bet != '');

SELECT geo FROM (SELECT MAX(bet), geo FROM LOG inner join USERS on LOG.user_id = USERS.user_id and bet != '');


--Для решения задачи необходимо выполнить запрос: 
SELECT AVG(Difference) from (SELECT user_id, (strftime('%s', min_time) - strftime('%s', MIN(time))) as Difference from (SELECT * from (LOG inner join (SELECT user_id, MIN(time) as min_time from LOG where bet != '' GROUP BY user_id) as A on LOG.user_id=A.user_id)) GROUP BY user_id)